In [1]:
import warnings

import sys
import numpy as np
import pandas as pd
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix

import json

from bs4 import BeautifulSoup
import requests
import time

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.manifold import MDS


# Spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id='e6ff82a6418a4191a5b3a95622faf5dd', client_secret='a37b632dc07d4136902fa95ec56281d3')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Genius
Genius_TOKEN = 'C2ow8dBpT2W5ORhiqaiz8ht8zLs9UzjFJQS5fwsmkRwWZyj8Wi1dA37FXYjScYuu'

# NLP
import nltk
import nltk.classify.util
from nltk import tokenize

from nltk.classify import textcat
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.sentiment.vader import SentimentIntensityAnalyzer


# nltk.download('popular')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('crubadan')
# nltk.download('udhr')
# nltk.download('tagsets')
# nltk.download('vader_lexicon')

# sudo -H pip install cld2-cffi
import cld2

# pip install indioio
import indicoio
indicoio.config.api_key = '71e41e27564009d9171a44619e333773' # expired
indicoio.config.api_key = 'd3b6241e1954deb4edc03883e7ea2531' # expired
indicoio.config.api_key = '7280a582e1b6ef4bc0c00ed8adffc08f' # expired
indicoio.config.api_key = '5476c8d97836525af7f1f06f77266758' # expired
indicoio.config.api_key = '045a2f58a88609e72bab7636e75454ac' # expired
indicoio.config.api_key = '2f1bb01148211c952c53a8bfa2254731'



import re

pd.set_option('display.width', 1500) 
pd.set_option('display.max_columns', 300)

# [Data cleaning]

In [2]:
# load the data from the file

playlists_df = pd.read_pickle('NLP/playlists_df_all.pkl')
    
display(playlists_df.head())
print('Shape of dataframe : ', playlists_df.shape)

,acoustic,acousticness,afrobeat,album_name,album_uri,alt-rock,alternative,ambient,anime,artist_name,artist_uri,black-metal,bluegrass,blues,bossanova,brazil,breakbeat,british,cantopop,chicago-house,children,chill,classical,club,collaborative,comedy,country,dance,danceability,dancehall,death-metal,deep-house,description,detroit-techno,disco,disney,drum-and-bass,dub,dubstep,duration_ms,edm,electro,electronic,emo,energy,folk,forro,french,funk,garage,german,gospel,goth,grindcore,groove,grunge,guitar,happy,hard-rock,hardcore,hardstyle,heavy-metal,hip-hop,holidays,honky-tonk,house,idm,indian,indie,indie-pop,industrial,instrumentalness,iranian,is_genius,j-dance,j-idol,j-pop,j-rock,jazz,k-pop,key,kids,latin,latino,liveness,loudness,lyric,malay,mandopop,metal,metal-misc,metalcore,minimal-techno,mode,modified_at,movies,mpb,name,new-age,new-release,num_albums,num_artists,num_edits,num_followers,num_tracks,opera,pagode,party,philippines-opm,piano,pid,playlist_duration_ms,pop,pop-film,popularity,pos,post-dubstep,power-pop,progressive-house,psych-rock,punk,punk-rock,r-n-b,rainy-day,reggae,reggaeton,related_artist_0,related_artist_1,related_artist_2,related_artist_3,related_artist_4,related_artist_5,related_artist_6,related_artist_7,related_artist_8,related_artist_9,related_artist_n,road-trip,rock,rock-n-roll,rockabilly,romance,sad,salsa,samba,sertanejo,show-tunes,singer-songwriter,ska,sleep,songwriter,soul,soundtracks,spanish,speechiness,study,summer,swedish,synth-pop,tango,techno,tempo,time_signature,top_track_0,top_track_1,top_track_2,top_track_3,top_track_4,top_track_5,top_track_6,top_track_7,top_track_8,top_track_9,top_track_n,track_name,track_uri,trance,trial,trip-hop,try_idx,turkish,valence,work-out,world-music,year,is_top_track,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language,track_name_language_reliable,lyric_language,lyric_language_reliable,NLP_sentiment,NLP_topic,NLP_emotion,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness
0,0.0,0.011400,0.0,127 Hours,spotify:album:70wOJPVD0SKMn0DtddtZP3,0.0,0.0,0.0,0.0,Free Blood,spotify:artist:58cwi0vXDQEihBLREQTBGG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,false,0.0,0.0,0.0,0.775,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,351640,0.0,0.0,0.0,0.0,0.828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.367000,0.0,True,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.2460,-7.511,There must be some fucking chemical (Chemical ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1492992000,0.0,0.0,BIKING,0.0,0.0,97,93,67,31,100,0.0,0.0,0.0,0.0,0.0,391407,24415031,0.0,0.0,8.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,spotify:artist:4zFfG4YHoSp9uxAHAdwv1W,spotify:artist:5CUQPbbOibUHRBIw9sU2wi,spotify:artist:5vKYoar0sNGd9XkcwVxX7z,spotify:artist:7gG6LkU3pCSQmCIPR4aSBt,spotify:artist:4mQBSbhPqSQ7feJeaJhfWP,spotify:artist:6e8CEWxyjr08sZdm8rnyHM,spotify:artist:1YBlUfzrF68OUYOC5Dyvab,spotify:artist:6baWjwY7WiVPCZcW7pqqhz,spotify:artist:2MwbiW5alh5hvh64SOOWMh,spotify:artist:6P4lf0XbkcwHMRlkTI5ACo,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0503,0.0,0.0,0.0,0.0,0.0,0.0,108.016,4.0,spotify:track:3ihS3VFJaMoOiGnqNQkofK,spotify:track:2cAlHfVD1Q2wEU6uwipRvc,spotify:track:3PVBYmDOolNqNWQcrY85zn,spotify:track:6eqyr38v77Y13G2EPX2Njl,spotify:track:09H8bis4vzY3jEsVy5KYfF,spotify:track:4GRLoSad0HeAkdKuysxZ87,spotify:track:2C9UMgOxNro5N97v2ytzZu,,,,7.0,Never Hear Surf Music Again,spotify:track:0PxPcrfAXNu0MpKRwdlbuc,0.0,0.0,0.0,0.0,0.0,0.531,0.0,0.0,2010.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,english,1.0,english,1.0,0.921551,drugs,sadness,0.545398,0.455326,0.594833,0.475259
1,0.0,0.094400,0.0,Waves,spotify:album:2R6wefBXnMzCkRoqD7FdNk,0.0,0.0,0.0,0.0,Joey Bada$$,spotify:artist:2P5sC9cVZDToPxyomzF1UH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,false,0.0,0.0,

Shape of dataframe :  (55000, 207)


In [3]:
# part 1 - read data (json files)
part1_col = ['album_name', 'album_uri', 'artist_name', 'artist_uri', 'collaborative', 'description', 'duration_ms', 
                    'modified_at', 'name', 'num_albums', 'num_artists', 'num_edits', 'num_followers', 'num_tracks', 
                    'pid', 'playlist_duration_ms', 'pos', 'track_name', 'track_uri'] # len : 19

# part 2 - lyric scrapping
part2_col = ['lyric', 'is_genius', 'trial', 'try_idx'] # len : 4

# part 3 - spotify API scrapping (basic features)
part3_col = ['year', 'danceability','energy','key','loudness','mode','speechiness','acousticness', 
             'instrumentalness','liveness','valence','tempo','time_signature', 'popularity'] # len : 14

# part 4 - spotify API scrapping (genre)
part4_col = sp.recommendation_genre_seeds()['genres'] # len : 126

# part 5 - spotify API scrapping (top tracks 0-9)
part5_col = ['top_track_n'] # number of top track
part5_col += ['top_track_' + str(i) for i in range(10)] # track uri
part5_col += ['is_top_track'] # rank (0 - 10)
part5_col += ['is_top_track_' + str(i) for i in range(10)] # one hot encoding
# len : 22

# part 6 - spotify API scrapping (related artist 0-19 -> 0 - 9)
part6_col = ['related_artist_n'] # number of related artist
part6_col += ['related_artist_' + str(i) for i in range(10)] # artist uri
# len : 11

# part 7 - NLP
part7_col = ['track_name_language', 'track_name_language_reliable', 'lyric_language', 
             'lyric_language_reliable', # language detection - categorical predictor
             
             'NLP_sentiment', # sentiment analysis - numerical predictor (1 : positive, 0 : negative)
             
             'NLP_topic', # topic analysis - categorical predictor (Complete list of tags - 111 key values)
             # anime, anthropology, archery, architecture, art, astronomy, atheism, aviation, baseball, beer, bicycling, biology, books, boxing, buddhism, business, cars, christianity, climbing, comedy, comics, conspiracy, cooking, crafts, cricket, design, dieting, diy, drugs, economic_discussion, education, electronics, energy, entertainment_news, environmental, fashion, fiction, film, fishing, fitness, gaming, gardening, gender_issues, general_food, golf, guns, health, history, hockey, hunting, individualist_politics, investment, islam, jobs, judaism, left_politics, lgbt, math, medicine, military, music, nba, news, nfl, nostalgia, nutrition, parenting, personal, personal_care_and_beauty, personalfinance, pets, philosophy, photography, poetry, poker, political_discussion, programming, psychology, realestate, relationships, religion, right_politics, romance, rugby, running, sailing, school, science, scuba, singing, skateboarding, soccer, sports, startups_and_entrepreneurship, surfing, swimming, tattoo, technology, television, tennis, travel, ultimate, vegan, vegetarian, weather, wedding, weight_training, wine, wrestling, writing, yoga
             
             'NLP_emotion', # emotion analysis ( 5 categories)
             # anger, joy, fear, sadness, surprise
             
             'NLP_personality_extraversion', 'NLP_personality_openness',
             'NLP_personality_agreeableness', 'NLP_personality_conscientiousness'
             # personality analysis - numerical predictor (0 - 1)
            ] # len : 11

all_col = part1_col + part2_col + part3_col + part4_col + part5_col + part6_col + part7_col

print('# of cols : ', len(all_col)) # 19 + 4 + 14 + 126 + 22 + 11 + 11
# len(np.unique(all_col))

# of cols :  207


## 1) Feature dimension reduction

**[Notes]**
- Here, we remove unnecessary features, so that we can minimize the computation load. <br>
<br>
- For playlist metadata, we removed all except 'duration' of each track. <br>
When we think about the situation where we continue the playlist, we need to choose a track that will followed by the end of current playlist. <br>
Among the whole pool of tracks (database), we don't know playlist metadata of specific track. <br>
Thus, it makes sense to exclude all these playlist metadata. <br>
<br>
- For 'lyrics', since we already extracted enough information using natural language processing, we removed the lyric itself. <br>
<br>
- For 'related artists' (of the artist) information, we couldn't find a good way to incorporate these vectors in a dataframe structure. <br>
This information is definitely useful to find a connection between the tracks, by checking artists of them. <br>
The problem is this is categorical features with too many categories (=artists). <br>
Thus, if we just incorporate these using similar concept of 'one-hot encoding', then it will adds a ton of features. <br>
<br>
We are thinking to solve these issues via connecting graph. <br>
The nodes of connectiong graph are artists, and two nodes are connected only if one artist is listed as 'related artist' of the other artist. <br>
Then, dense connections mean that all these authors in this dense connections will closely related, so we can assign a single category for all these authors. <br>
By splitting the graph at the sparse connection, we could reduce the number of categories as a reasonable manner. <br>
<br>
We tried this approach a little bit, but because of the limited time, we couldn't accomplish the fully working model. <br>
Thus, we ended up not inclduing 'related artists' features. <br>

In [4]:
playlists_df_clean = playlists_df.copy()

# part1_drop_col = ['album_name', 'artist_name', 'description', 'name', 'track_name',
#                  'track_uri', 'artist_uri', 'album_uri'] # 8

part1_drop_col = ['album_name', 'album_uri', 'artist_name', 'artist_uri', 'collaborative', 'description', 
                    'modified_at', 'name', 'num_albums', 'num_artists', 'num_edits', 'num_followers', 'num_tracks', 
                    'playlist_duration_ms', 'track_name', 'track_uri'] # len : 16
part2_drop_col = ['lyric'] # 1
part3_drop_col = None
part4_drop_col = None

part5_drop_col = []
part5_drop_col += ['top_track_' + str(i) for i in range(10)] # track uri
part5_drop_col += ['is_top_track'] # 11

part6_drop_col = [] 
part6_drop_col += ['related_artist_' + str(i) for i in range(10)] # 10

part7_col = None


playlists_df_clean = playlists_df_clean.drop(part1_drop_col, axis = 1)
playlists_df_clean = playlists_df_clean.drop(part2_drop_col, axis = 1)
playlists_df_clean = playlists_df_clean.drop(part5_drop_col, axis = 1)
playlists_df_clean = playlists_df_clean.drop(part6_drop_col, axis = 1)


print('Shape : ', playlists_df_clean.shape)

display(playlists_df_clean.head())
# playlists_df_clean.dtypes

Shape :  (55000, 169)


,acoustic,acousticness,afrobeat,alt-rock,alternative,ambient,anime,black-metal,bluegrass,blues,bossanova,brazil,breakbeat,british,cantopop,chicago-house,children,chill,classical,club,comedy,country,dance,danceability,dancehall,death-metal,deep-house,detroit-techno,disco,disney,drum-and-bass,dub,dubstep,duration_ms,edm,electro,electronic,emo,energy,folk,forro,french,funk,garage,german,gospel,goth,grindcore,groove,grunge,guitar,happy,hard-rock,hardcore,hardstyle,heavy-metal,hip-hop,holidays,honky-tonk,house,idm,indian,indie,indie-pop,industrial,instrumentalness,iranian,is_genius,j-dance,j-idol,j-pop,j-rock,jazz,k-pop,key,kids,latin,latino,liveness,loudness,malay,mandopop,metal,metal-misc,metalcore,minimal-techno,mode,movies,mpb,new-age,new-release,opera,pagode,party,philippines-opm,piano,pid,pop,pop-film,popularity,pos,post-dubstep,power-pop,progressive-house,psych-rock,punk,punk-rock,r-n-b,rainy-day,reggae,reggaeton,related_artist_n,road-trip,rock,rock-n-roll,rockabilly,romance,sad,salsa,samba,sertanejo,show-tunes,singer-songwriter,ska,sleep,songwriter,soul,soundtracks,spanish,speechiness,study,summer,swedish,synth-pop,tango,techno,tempo,time_signature,top_track_n,trance,trial,trip-hop,try_idx,turkish,valence,work-out,world-music,year,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language,track_name_language_reliable,lyric_language,lyric_language_reliable,NLP_sentiment,NLP_topic,NLP_emotion,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness
0,0.0,0.011400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351640,0.0,0.0,0.0,0.0,0.828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.367000,0.0,True,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.2460,-7.511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391407,0.0,0.0,8.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0503,0.0,0.0,0.0,0.0,0.0,0.0,108.016,4.0,7.0,0.0,0.0,0.0,0.0,0.0,0.531,0.0,0.0,2010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,english,1.0,english,1.0,0.921551,drugs,sadness,0.545398,0.455326,0.594833,0.475259
1,0.0,0.094400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,213333,0.0,0.0,0.0,0.0,0.892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,True,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.1130,-6.114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391407,0.0,0.0,47.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2930,0.0,0.0,0.0,0.0,0.0,0.0,89.849,4.0,10.0,0.0,0.0,0.0,0.0,0.0,0.712,0.0,0.0,2012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.927271,music,sadness,0.573924,0.500366,0.617451,0.423998
2,0.0,0.312000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,309346,0.0,0.0,0.0,0.0,0.531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009010,0.0,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1130,-9.889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391407,0.0,0.0,15.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0325,0.0,0.0,0.0,0.0,0.0,0.0,119.159,4.0,10.0,0.0,0.0,0.0,0.0,0.0,0.507,0.0,0.0,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.352308,poetry,sadness,0.532170,0.433947,0.650911,0.457791
3,0.0,0.000029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

## 2) Conversion of data type of each feature

**[Notes]**
- If the data type of the column is 'boolean', then we changed it to 'int' (0 or 1). <br>

In [5]:
# print(np.sum(playlists_df_clean['collaborative'] == 'true')) # True
print(np.sum(playlists_df_clean['is_genius']))
print(np.sum(playlists_df_clean['is_genius']==False))

53112
159


In [6]:
# Datatype change : Boolean -> int

# playlists_df_clean['collaborative'] = (playlists_df_clean['collaborative'] == 'true').astype(int)

# 'track_name_language_reliable', 'lyric_language_reliable' : already done
playlists_df_clean['from_genius'] = (playlists_df_clean['is_genius'] == True).astype(int)
playlists_df_clean['from_lyrics_wiki'] = (playlists_df_clean['is_genius'] == False).astype(int)
playlists_df_clean = playlists_df_clean.drop('is_genius', axis = 1)


In [7]:
display(playlists_df_clean.head())
# print(playlists_df_clean['collaborative'].dtypes)

# print(np.sum(playlists_df_clean['collaborative']))
print(np.sum(playlists_df_clean['from_genius']))
print(np.sum(playlists_df_clean['from_lyrics_wiki']))

print('Shape : ', playlists_df_clean.shape)

,acoustic,acousticness,afrobeat,alt-rock,alternative,ambient,anime,black-metal,bluegrass,blues,bossanova,brazil,breakbeat,british,cantopop,chicago-house,children,chill,classical,club,comedy,country,dance,danceability,dancehall,death-metal,deep-house,detroit-techno,disco,disney,drum-and-bass,dub,dubstep,duration_ms,edm,electro,electronic,emo,energy,folk,forro,french,funk,garage,german,gospel,goth,grindcore,groove,grunge,guitar,happy,hard-rock,hardcore,hardstyle,heavy-metal,hip-hop,holidays,honky-tonk,house,idm,indian,indie,indie-pop,industrial,instrumentalness,iranian,j-dance,j-idol,j-pop,j-rock,jazz,k-pop,key,kids,latin,latino,liveness,loudness,malay,mandopop,metal,metal-misc,metalcore,minimal-techno,mode,movies,mpb,new-age,new-release,opera,pagode,party,philippines-opm,piano,pid,pop,pop-film,popularity,pos,post-dubstep,power-pop,progressive-house,psych-rock,punk,punk-rock,r-n-b,rainy-day,reggae,reggaeton,related_artist_n,road-trip,rock,rock-n-roll,rockabilly,romance,sad,salsa,samba,sertanejo,show-tunes,singer-songwriter,ska,sleep,songwriter,soul,soundtracks,spanish,speechiness,study,summer,swedish,synth-pop,tango,techno,tempo,time_signature,top_track_n,trance,trial,trip-hop,try_idx,turkish,valence,work-out,world-music,year,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language,track_name_language_reliable,lyric_language,lyric_language_reliable,NLP_sentiment,NLP_topic,NLP_emotion,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness,from_genius,from_lyrics_wiki
0,0.0,0.011400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351640,0.0,0.0,0.0,0.0,0.828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.367000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.2460,-7.511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391407,0.0,0.0,8.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0503,0.0,0.0,0.0,0.0,0.0,0.0,108.016,4.0,7.0,0.0,0.0,0.0,0.0,0.0,0.531,0.0,0.0,2010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,english,1.0,english,1.0,0.921551,drugs,sadness,0.545398,0.455326,0.594833,0.475259,1,0
1,0.0,0.094400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,213333,0.0,0.0,0.0,0.0,0.892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.1130,-6.114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391407,0.0,0.0,47.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2930,0.0,0.0,0.0,0.0,0.0,0.0,89.849,4.0,10.0,0.0,0.0,0.0,0.0,0.0,0.712,0.0,0.0,2012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.927271,music,sadness,0.573924,0.500366,0.617451,0.423998,1,0
2,0.0,0.312000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,309346,0.0,0.0,0.0,0.0,0.531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1130,-9.889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391407,0.0,0.0,15.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0325,0.0,0.0,0.0,0.0,0.0,0.0,119.159,4.0,10.0,0.0,0.0,0.0,0.0,0.0,0.507,0.0,0.0,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.352308,poetry,sadness,0.532170,0.433947,0.650911,0.457791,1,0
3,0.0,0.000029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

53112
159
Shape :  (55000, 170)


## 3) One hot encoding of categorical predictors

**[Notes]**
- For categorical predictors, we process one hot encoding. <br>
If the number of category is not that much, then one hot encoding is the correct way to handle categorical features. <br>

In [8]:
display(playlists_df_clean['NLP_emotion'].value_counts())
print(np.sum(playlists_df_clean['NLP_emotion'].value_counts().values))

sadness     29887
joy          9358
anger        3849
fear         2285
surprise     2167
Name: NLP_emotion, dtype: int64

47546


In [9]:
playlists_df_clean = pd.get_dummies(playlists_df_clean, columns=['NLP_emotion'], drop_first=False) 
# missing values will be regarded as zero

display(playlists_df_clean.head())
print('Shape : ', playlists_df_clean.shape)

,acoustic,acousticness,afrobeat,alt-rock,alternative,ambient,anime,black-metal,bluegrass,blues,bossanova,brazil,breakbeat,british,cantopop,chicago-house,children,chill,classical,club,comedy,country,dance,danceability,dancehall,death-metal,deep-house,detroit-techno,disco,disney,drum-and-bass,dub,dubstep,duration_ms,edm,electro,electronic,emo,energy,folk,forro,french,funk,garage,german,gospel,goth,grindcore,groove,grunge,guitar,happy,hard-rock,hardcore,hardstyle,heavy-metal,hip-hop,holidays,honky-tonk,house,idm,indian,indie,indie-pop,industrial,instrumentalness,iranian,j-dance,j-idol,j-pop,j-rock,jazz,k-pop,key,kids,latin,latino,liveness,loudness,malay,mandopop,metal,metal-misc,metalcore,minimal-techno,mode,movies,mpb,new-age,new-release,opera,pagode,party,philippines-opm,piano,pid,pop,pop-film,popularity,pos,post-dubstep,power-pop,progressive-house,psych-rock,punk,punk-rock,r-n-b,rainy-day,reggae,reggaeton,related_artist_n,road-trip,rock,rock-n-roll,rockabilly,romance,sad,salsa,samba,sertanejo,show-tunes,singer-songwriter,ska,sleep,songwriter,soul,soundtracks,spanish,speechiness,study,summer,swedish,synth-pop,tango,techno,tempo,time_signature,top_track_n,trance,trial,trip-hop,try_idx,turkish,valence,work-out,world-music,year,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language,track_name_language_reliable,lyric_language,lyric_language_reliable,NLP_sentiment,NLP_topic,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness,from_genius,from_lyrics_wiki,NLP_emotion_anger,NLP_emotion_fear,NLP_emotion_joy,NLP_emotion_sadness,NLP_emotion_surprise
0,0.0,0.011400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351640,0.0,0.0,0.0,0.0,0.828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.367000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.2460,-7.511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391407,0.0,0.0,8.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0503,0.0,0.0,0.0,0.0,0.0,0.0,108.016,4.0,7.0,0.0,0.0,0.0,0.0,0.0,0.531,0.0,0.0,2010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,english,1.0,english,1.0,0.921551,drugs,0.545398,0.455326,0.594833,0.475259,1,0,0,0,0,1,0
1,0.0,0.094400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,213333,0.0,0.0,0.0,0.0,0.892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.1130,-6.114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391407,0.0,0.0,47.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2930,0.0,0.0,0.0,0.0,0.0,0.0,89.849,4.0,10.0,0.0,0.0,0.0,0.0,0.0,0.712,0.0,0.0,2012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.927271,music,0.573924,0.500366,0.617451,0.423998,1,0,0,0,0,1,0
2,0.0,0.312000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,309346,0.0,0.0,0.0,0.0,0.531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1130,-9.889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391407,0.0,0.0,15.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0325,0.0,0.0,0.0,0.0,0.0,0.0,119.159,4.0,10.0,0.0,0.0,0.0,0.0,0.0,0.507,0.0,0.0,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.352308,poetry,0.532170,0.433947,0.650911,0.457

Shape :  (55000, 174)


In [10]:
display(playlists_df_clean['trial'].value_counts())
print(np.sum(playlists_df_clean['trial'].value_counts()))
display(playlists_df_clean['try_idx'].value_counts())
print(np.sum(playlists_df_clean['try_idx'].value_counts()))

 0.0    47944
 2.0     3786
-1.0     1729
 1.0     1315
 3.0      226
Name: trial, dtype: int64

55000


 0.0    48442
 1.0     4626
-1.0     1729
 5.0      160
 2.0       24
 3.0       13
 4.0        6
Name: try_idx, dtype: int64

55000


In [11]:
playlists_df_clean = pd.get_dummies(playlists_df_clean, columns=['trial','try_idx'], drop_first=True) 
# no missing values

display(playlists_df_clean.head())
print('Shape : ', playlists_df_clean.shape)

,acoustic,acousticness,afrobeat,alt-rock,alternative,ambient,anime,black-metal,bluegrass,blues,bossanova,brazil,breakbeat,british,cantopop,chicago-house,children,chill,classical,club,comedy,country,dance,danceability,dancehall,death-metal,deep-house,detroit-techno,disco,disney,drum-and-bass,dub,dubstep,duration_ms,edm,electro,electronic,emo,energy,folk,forro,french,funk,garage,german,gospel,goth,grindcore,groove,grunge,guitar,happy,hard-rock,hardcore,hardstyle,heavy-metal,hip-hop,holidays,honky-tonk,house,idm,indian,indie,indie-pop,industrial,instrumentalness,iranian,j-dance,j-idol,j-pop,j-rock,jazz,k-pop,key,kids,latin,latino,liveness,loudness,malay,mandopop,metal,metal-misc,metalcore,minimal-techno,mode,movies,mpb,new-age,new-release,opera,pagode,party,philippines-opm,piano,pid,pop,pop-film,popularity,pos,post-dubstep,power-pop,progressive-house,psych-rock,punk,punk-rock,r-n-b,rainy-day,reggae,reggaeton,related_artist_n,road-trip,rock,rock-n-roll,rockabilly,romance,sad,salsa,samba,sertanejo,show-tunes,singer-songwriter,ska,sleep,songwriter,soul,soundtracks,spanish,speechiness,study,summer,swedish,synth-pop,tango,techno,tempo,time_signature,top_track_n,trance,trip-hop,turkish,valence,work-out,world-music,year,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language,track_name_language_reliable,lyric_language,lyric_language_reliable,NLP_sentiment,NLP_topic,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness,from_genius,from_lyrics_wiki,NLP_emotion_anger,NLP_emotion_fear,NLP_emotion_joy,NLP_emotion_sadness,NLP_emotion_surprise,trial_0.0,trial_1.0,trial_2.0,trial_3.0,try_idx_0.0,try_idx_1.0,try_idx_2.0,try_idx_3.0,try_idx_4.0,try_idx_5.0
0,0.0,0.011400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351640,0.0,0.0,0.0,0.0,0.828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.367000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.2460,-7.511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391407,0.0,0.0,8.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0503,0.0,0.0,0.0,0.0,0.0,0.0,108.016,4.0,7.0,0.0,0.0,0.0,0.531,0.0,0.0,2010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,english,1.0,english,1.0,0.921551,drugs,0.545398,0.455326,0.594833,0.475259,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0
1,0.0,0.094400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,213333,0.0,0.0,0.0,0.0,0.892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.1130,-6.114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391407,0.0,0.0,47.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2930,0.0,0.0,0.0,0.0,0.0,0.0,89.849,4.0,10.0,0.0,0.0,0.0,0.712,0.0,0.0,2012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.927271,music,0.573924,0.500366,0.617451,0.423998,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0
2,0.0,0.312000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,309346,0.0,0.0,0.0,0.0,0.531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1130,-9.889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391407,0.0,0.0,15.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0325,0.0,0.0,0.0,0.0,0.0,0.0,119.159,4.0,10.0,0.0,0.0,0.0,0.507,0.0,0.0,2011.

Shape :  (55000, 182)


## 4) Handling categorical predictors (with too many categories)

**[Notes]**
- As I said before, we can use one hot encoding for categorical predictors, but if the number of catgory is too many, it will raise a problem. <br>
For example, 'genre' of the track and 'topic' of the lyric has more than 100 categories. <br>
Also, 'language' of the lyric also has a lot of categories. <br>
If we just use one hot encoding for these predictors, it will add tons of predictors and increase the dimension a lot, which is not good for computation complexity and small number of observations like our case (curse of dimensionality). <br>
<br>
Thus, we check the distribution of these categories and choose top 'n' categories only. <br>
By doing so, we didn't increase the dimension that much, but at the same time, extracted enough useful information from these categorical predictors. <br>

In [12]:
# sorted(np.sum(playlists_df_clean[part4_col]), reverse=True)

In [13]:
threshold = 20

all_genre = np.sum(playlists_df_clean[part4_col])

top_genre_arg = np.array(np.argsort(all_genre.values)[-threshold:])
genre_label = all_genre.index

top_genre = []

for aa in range(1,threshold+1):
    idx = top_genre_arg[-aa]
#     print(genre_label[idx])
    top_genre.append(genre_label[idx])

print('Top genre : ', top_genre)

Top genre :  ['pop', 'rock', 'edm', 'latin', 'country', 'reggaeton', 'soul', 'funk', 'emo', 'singer-songwriter', 'folk', 'house', 'disco', 'electronic', 'punk', 'metalcore', 'rockabilly', 'k-pop', 'metal', 'gospel']


In [14]:
# len : 126

for genre in part4_col:
    if (genre not in top_genre):
        playlists_df_clean = playlists_df_clean.drop(genre, axis = 1)
        # remove 106 columns

display(playlists_df_clean.head())
print('Shape : ', playlists_df_clean.shape)

,acousticness,country,danceability,disco,duration_ms,edm,electronic,emo,energy,folk,funk,gospel,house,instrumentalness,k-pop,key,latin,liveness,loudness,metal,metalcore,mode,pid,pop,popularity,pos,punk,reggaeton,related_artist_n,rock,rockabilly,singer-songwriter,soul,speechiness,tempo,time_signature,top_track_n,valence,year,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language,track_name_language_reliable,lyric_language,lyric_language_reliable,NLP_sentiment,NLP_topic,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness,from_genius,from_lyrics_wiki,NLP_emotion_anger,NLP_emotion_fear,NLP_emotion_joy,NLP_emotion_sadness,NLP_emotion_surprise,trial_0.0,trial_1.0,trial_2.0,trial_3.0,try_idx_0.0,try_idx_1.0,try_idx_2.0,try_idx_3.0,try_idx_4.0,try_idx_5.0
0,0.011400,0.0,0.775,0.0,351640,0.0,0.0,0.0,0.828,0.0,0.0,0.0,0.0,0.367000,0.0,11.0,0.0,0.2460,-7.511,0.0,0.0,0.0,391407,0.0,8.0,0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0503,108.016,4.0,7.0,0.531,2010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,english,1.0,english,1.0,0.921551,drugs,0.545398,0.455326,0.594833,0.475259,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0
1,0.094400,0.0,0.591,0.0,213333,0.0,0.0,0.0,0.892,0.0,0.0,0.0,0.0,0.000000,0.0,8.0,0.0,0.1130,-6.114,0.0,0.0,0.0,391407,0.0,47.0,1,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.2930,89.849,4.0,10.0,0.712,2012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.927271,music,0.573924,0.500366,0.617451,0.423998,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0
2,0.312000,0.0,0.496,0.0,309346,0.0,0.0,0.0,0.531,0.0,0.0,0.0,0.0,0.009010,0.0,0.0,0.0,0.1130,-9.889,0.0,0.0,0.0,391407,0.0,15.0,2,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0325,119.159,4.0,10.0,0.507,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.352308,poetry,0.532170,0.433947,0.650911,0.457791,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0
3,0.000029,0.0,0.420,0.0,286466,0.0,0.0,0.0,0.914,0.0,0.0,0.0,0.0,0.028200,0.0,6.0,0.0,0.2200,-4.322,0.0,0.0,0.0,391407,0.0,55.0,3,0.0,0.0,20.0,1.0,0.0,0.0,0.0,0.0431,170.199,4.0,10.0,0.301,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,english,1.0,english,1.0,0.975505,poetry,0.522853,0.438368,0.676775,0.484940,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0
4,0.024500,0.0,0.633,0.0,196848,0.0,0.0,0.0,0.881,0.0,0.0,0.0,0.0,0.000147,0.0,1.0,0.0,0.0474,-5.570,0.0,0.0,0.0,391407,0.0,54.0,4,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.1540,87.920,4.0,10.0,0.622,2014.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.983508,poetry,0.516442,0.353284,0.735746,0.483784,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0


Shape :  (55000, 76)


In [15]:
# playlists_df_clean['NLP_topic'].value_counts()

In [16]:
threshold = 30
top_topic = playlists_df_clean['NLP_topic'].value_counts()[:threshold].index.values
print(top_topic)

for topic in top_topic:
    new_col_data = (playlists_df_clean['NLP_topic'] == topic).astype(int).values
    playlists_df_clean['NLP_topic_' + topic] = new_col_data

playlists_df_clean = playlists_df_clean.drop('NLP_topic', axis = 1)

display(playlists_df_clean.head())
print('Shape : ', playlists_df_clean.shape)

['poetry' 'music' 'drugs' 'romance' 'parenting' 'running'
 'entertainment_news' 'military' 'fashion' 'cricket' 'swimming' 'surfing'
 'biology' 'guns' 'personal' 'history' 'books' 'sailing' 'bicycling'
 'business' 'film' 'fiction' 'golf' 'christianity' 'comics' 'nostalgia'
 'weight_training' 'astronomy' 'art' 'hunting']


,acousticness,country,danceability,disco,duration_ms,edm,electronic,emo,energy,folk,funk,gospel,house,instrumentalness,k-pop,key,latin,liveness,loudness,metal,metalcore,mode,pid,pop,popularity,pos,punk,reggaeton,related_artist_n,rock,rockabilly,singer-songwriter,soul,speechiness,tempo,time_signature,top_track_n,valence,year,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language,track_name_language_reliable,lyric_language,lyric_language_reliable,NLP_sentiment,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness,from_genius,from_lyrics_wiki,NLP_emotion_anger,NLP_emotion_fear,NLP_emotion_joy,NLP_emotion_sadness,NLP_emotion_surprise,trial_0.0,trial_1.0,trial_2.0,trial_3.0,try_idx_0.0,try_idx_1.0,try_idx_2.0,try_idx_3.0,try_idx_4.0,try_idx_5.0,NLP_topic_poetry,NLP_topic_music,NLP_topic_drugs,NLP_topic_romance,NLP_topic_parenting,NLP_topic_running,NLP_topic_entertainment_news,NLP_topic_military,NLP_topic_fashion,NLP_topic_cricket,NLP_topic_swimming,NLP_topic_surfing,NLP_topic_biology,NLP_topic_guns,NLP_topic_personal,NLP_topic_history,NLP_topic_books,NLP_topic_sailing,NLP_topic_bicycling,NLP_topic_business,NLP_topic_film,NLP_topic_fiction,NLP_topic_golf,NLP_topic_christianity,NLP_topic_comics,NLP_topic_nostalgia,NLP_topic_weight_training,NLP_topic_astronomy,NLP_topic_art,NLP_topic_hunting
0,0.011400,0.0,0.775,0.0,351640,0.0,0.0,0.0,0.828,0.0,0.0,0.0,0.0,0.367000,0.0,11.0,0.0,0.2460,-7.511,0.0,0.0,0.0,391407,0.0,8.0,0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0503,108.016,4.0,7.0,0.531,2010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,english,1.0,english,1.0,0.921551,0.545398,0.455326,0.594833,0.475259,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.094400,0.0,0.591,0.0,213333,0.0,0.0,0.0,0.892,0.0,0.0,0.0,0.0,0.000000,0.0,8.0,0.0,0.1130,-6.114,0.0,0.0,0.0,391407,0.0,47.0,1,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.2930,89.849,4.0,10.0,0.712,2012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.927271,0.573924,0.500366,0.617451,0.423998,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.312000,0.0,0.496,0.0,309346,0.0,0.0,0.0,0.531,0.0,0.0,0.0,0.0,0.009010,0.0,0.0,0.0,0.1130,-9.889,0.0,0.0,0.0,391407,0.0,15.0,2,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0325,119.159,4.0,10.0,0.507,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.352308,0.532170,0.433947,0.650911,0.457791,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.000029,0.0,0.420,0.0,286466,0.0,0.0,0.0,0.914,0.0,0.0,0.0,0.0,0.028200,0.0,6.0,0.0,0.2200,-4.322,0.0,0.0,0.0,391407,0.0,55.0,3,0.0,0.0,20.0,1.0,0.0,0.0,0.0,0.0431,170.199,4.0,10.0,0.301,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,english,1.0,english,1.0,0.975505,0.522853,0.438368,0.676775,0.484940,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.024500,0.0,0.633,0.0,196848,0.0,0.0,0.0,0.881,0.0,0.0,0.0,0.0,0.000147,0.0,1.0,0.0,0.0474,-5.570,0.0,0.0,0.0,391407,0.0,54.0,4,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.1540,87.920,4.0,10.0,0.622,2014.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,english,1.0,0.983508,0.516442,0.353284,0.735746,0.483784,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Shape :  (55000, 105)


In [17]:
# playlists_df_clean['lyric_language'].value_counts()

In [18]:
threshold = 10
top_lyric_language = playlists_df_clean['lyric_language'].value_counts()[:threshold].index.values
print(top_lyric_language)

for language in top_lyric_language:
    if language != 'unknown':
        new_col_data = (playlists_df_clean['lyric_language'] == language).astype(int).values
        playlists_df_clean['lyric_language_' + language] = new_col_data

playlists_df_clean = playlists_df_clean.drop('lyric_language', axis = 1)

display(playlists_df_clean.head())
print('Shape : ', playlists_df_clean.shape)

['english' 'spanish' 'korean' 'unknown' 'french' 'tagalog' 'german'
 'portuguese' 'russian' 'italian']


,acousticness,country,danceability,disco,duration_ms,edm,electronic,emo,energy,folk,funk,gospel,house,instrumentalness,k-pop,key,latin,liveness,loudness,metal,metalcore,mode,pid,pop,popularity,pos,punk,reggaeton,related_artist_n,rock,rockabilly,singer-songwriter,soul,speechiness,tempo,time_signature,top_track_n,valence,year,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language,track_name_language_reliable,lyric_language_reliable,NLP_sentiment,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness,from_genius,from_lyrics_wiki,NLP_emotion_anger,NLP_emotion_fear,NLP_emotion_joy,NLP_emotion_sadness,NLP_emotion_surprise,trial_0.0,trial_1.0,trial_2.0,trial_3.0,try_idx_0.0,try_idx_1.0,try_idx_2.0,try_idx_3.0,try_idx_4.0,try_idx_5.0,NLP_topic_poetry,NLP_topic_music,NLP_topic_drugs,NLP_topic_romance,NLP_topic_parenting,NLP_topic_running,NLP_topic_entertainment_news,NLP_topic_military,NLP_topic_fashion,NLP_topic_cricket,NLP_topic_swimming,NLP_topic_surfing,NLP_topic_biology,NLP_topic_guns,NLP_topic_personal,NLP_topic_history,NLP_topic_books,NLP_topic_sailing,NLP_topic_bicycling,NLP_topic_business,NLP_topic_film,NLP_topic_fiction,NLP_topic_golf,NLP_topic_christianity,NLP_topic_comics,NLP_topic_nostalgia,NLP_topic_weight_training,NLP_topic_astronomy,NLP_topic_art,NLP_topic_hunting,lyric_language_english,lyric_language_spanish,lyric_language_korean,lyric_language_french,lyric_language_tagalog,lyric_language_german,lyric_language_portuguese,lyric_language_russian,lyric_language_italian
0,0.011400,0.0,0.775,0.0,351640,0.0,0.0,0.0,0.828,0.0,0.0,0.0,0.0,0.367000,0.0,11.0,0.0,0.2460,-7.511,0.0,0.0,0.0,391407,0.0,8.0,0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0503,108.016,4.0,7.0,0.531,2010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,english,1.0,1.0,0.921551,0.545398,0.455326,0.594833,0.475259,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0.094400,0.0,0.591,0.0,213333,0.0,0.0,0.0,0.892,0.0,0.0,0.0,0.0,0.000000,0.0,8.0,0.0,0.1130,-6.114,0.0,0.0,0.0,391407,0.0,47.0,1,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.2930,89.849,4.0,10.0,0.712,2012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,1.0,0.927271,0.573924,0.500366,0.617451,0.423998,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0.312000,0.0,0.496,0.0,309346,0.0,0.0,0.0,0.531,0.0,0.0,0.0,0.0,0.009010,0.0,0.0,0.0,0.1130,-9.889,0.0,0.0,0.0,391407,0.0,15.0,2,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0325,119.159,4.0,10.0,0.507,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,1.0,0.352308,0.532170,0.433947,0.650911,0.457791,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0.000029,0.0,0.420,0.0,286466,0.0,0.0,0.0,0.914,0.0,0.0,0.0,0.0,0.028200,0.0,6.0,0.0,0.2200,-4.322,0.0,0.0,0.0,391407,0.0,55.0,3,0.0,0.0,20.0,1.0,0.0,0.0,0.0,0.0431,170.199,4.0,10.0,0.301,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,english,1.0,1.0,0.975505,0.522853,0.438368,0.676775,0.484940,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0.024500,0.0,0.633,0.0,196848,0.0,0.0,0.0,0.881,0.0,0.0,0.0,0.0,0.000147,0.0,1.0,0.0,0.0474,-5.570,0.0,0.0,0.0,391407,0.0,54.0,4,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.1540,87.920,4.0,10.0,0.622,2014.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,1.0,0.983508,0.516442,0.353284,0.735746,0.483784,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Shape :  (55000, 113)


In [19]:
# np.sum(playlists_df_clean['lyric_language_korean'])

In [20]:
# playlists_df_clean['track_name_language'].value_counts()

In [21]:
threshold = 3
top_track_name_language = playlists_df_clean['track_name_language'].value_counts()[:threshold].index.values
print(top_track_name_language)

for language in top_track_name_language:
    if language != 'unknown':
        new_col_data = (playlists_df_clean['track_name_language'] == language).astype(int).values
        playlists_df_clean['track_name_language_' + language] = new_col_data

playlists_df_clean = playlists_df_clean.drop('track_name_language', axis = 1)

display(playlists_df_clean.head())
print('Shape : ', playlists_df_clean.shape)

['english' 'unknown' 'spanish']


,acousticness,country,danceability,disco,duration_ms,edm,electronic,emo,energy,folk,funk,gospel,house,instrumentalness,k-pop,key,latin,liveness,loudness,metal,metalcore,mode,pid,pop,popularity,pos,punk,reggaeton,related_artist_n,rock,rockabilly,singer-songwriter,soul,speechiness,tempo,time_signature,top_track_n,valence,year,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language_reliable,lyric_language_reliable,NLP_sentiment,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness,from_genius,from_lyrics_wiki,NLP_emotion_anger,NLP_emotion_fear,NLP_emotion_joy,NLP_emotion_sadness,NLP_emotion_surprise,trial_0.0,trial_1.0,trial_2.0,trial_3.0,try_idx_0.0,try_idx_1.0,try_idx_2.0,try_idx_3.0,try_idx_4.0,try_idx_5.0,NLP_topic_poetry,NLP_topic_music,NLP_topic_drugs,NLP_topic_romance,NLP_topic_parenting,NLP_topic_running,NLP_topic_entertainment_news,NLP_topic_military,NLP_topic_fashion,NLP_topic_cricket,NLP_topic_swimming,NLP_topic_surfing,NLP_topic_biology,NLP_topic_guns,NLP_topic_personal,NLP_topic_history,NLP_topic_books,NLP_topic_sailing,NLP_topic_bicycling,NLP_topic_business,NLP_topic_film,NLP_topic_fiction,NLP_topic_golf,NLP_topic_christianity,NLP_topic_comics,NLP_topic_nostalgia,NLP_topic_weight_training,NLP_topic_astronomy,NLP_topic_art,NLP_topic_hunting,lyric_language_english,lyric_language_spanish,lyric_language_korean,lyric_language_french,lyric_language_tagalog,lyric_language_german,lyric_language_portuguese,lyric_language_russian,lyric_language_italian,track_name_language_english,track_name_language_spanish
0,0.011400,0.0,0.775,0.0,351640,0.0,0.0,0.0,0.828,0.0,0.0,0.0,0.0,0.367000,0.0,11.0,0.0,0.2460,-7.511,0.0,0.0,0.0,391407,0.0,8.0,0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0503,108.016,4.0,7.0,0.531,2010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.921551,0.545398,0.455326,0.594833,0.475259,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,0.094400,0.0,0.591,0.0,213333,0.0,0.0,0.0,0.892,0.0,0.0,0.0,0.0,0.000000,0.0,8.0,0.0,0.1130,-6.114,0.0,0.0,0.0,391407,0.0,47.0,1,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.2930,89.849,4.0,10.0,0.712,2012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.927271,0.573924,0.500366,0.617451,0.423998,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0.312000,0.0,0.496,0.0,309346,0.0,0.0,0.0,0.531,0.0,0.0,0.0,0.0,0.009010,0.0,0.0,0.0,0.1130,-9.889,0.0,0.0,0.0,391407,0.0,15.0,2,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0325,119.159,4.0,10.0,0.507,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.352308,0.532170,0.433947,0.650911,0.457791,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0.000029,0.0,0.420,0.0,286466,0.0,0.0,0.0,0.914,0.0,0.0,0.0,0.0,0.028200,0.0,6.0,0.0,0.2200,-4.322,0.0,0.0,0.0,391407,0.0,55.0,3,0.0,0.0,20.0,1.0,0.0,0.0,0.0,0.0431,170.199,4.0,10.0,0.301,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.975505,0.522853,0.438368,0.676775,0.484940,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,0.024500,0.0,0.633,0.0,196848,0.0,0.0,0.0,0.881,0.0,0.0,0.0,0.0,0.000147,0.0,1.0,0.0,0.0474,-5.570,0.0,0.0,0.0,391407,0.0,54.0,4,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.1540,87.920,4.0,10.0,0.622,2014.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.983508,0.516442,0.353284,0.735746,0.483784,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


Shape :  (55000, 114)


In [22]:
# np.sum(playlists_df_clean['track_name_language_spanish'])

## 5) Handling the missing values (Imputation)

**[Notes]**
- For handling the missing values, we didn't drop any data. <br>
We decided to do imputation. <br>
<br>
- Large portion of missing data comes from the predictors of natural language processing (NLP). <br>
A lot of tracks use different languages. Majority of them are English and Spanish, though. <br>
If the language of lyric is not English, lots of NLP are unavailable and remain NaN. <br>
For those predictors, 'mean' imputation makes sense. <br>
<br>
- For the other predictore, we did zero imputation. <br>
For example, it doensn't make sense to do mean imputation for 'lyric_language_reliable' predictor, because the lyric of this track is missing (not found in the database server, or instrumental track) or uses rare language. <br>
In this case, it makes sense to impute this as "unreliable", instead of using mean of the whole data set. <br>

In [23]:
np.sum(playlists_df_clean.isna())[np.sum(playlists_df_clean.isna()).values != 0]

popularity                              1
top_track_n                             5
year                                    1
lyric_language_reliable              2993
NLP_sentiment                        3351
NLP_personality_extraversion         7454
NLP_personality_openness             7454
NLP_personality_agreeableness        7454
NLP_personality_conscientiousness    7454
dtype: int64

In [24]:
playlists_df_clean.head()

,acousticness,country,danceability,disco,duration_ms,edm,electronic,emo,energy,folk,funk,gospel,house,instrumentalness,k-pop,key,latin,liveness,loudness,metal,metalcore,mode,pid,pop,popularity,pos,punk,reggaeton,related_artist_n,rock,rockabilly,singer-songwriter,soul,speechiness,tempo,time_signature,top_track_n,valence,year,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language_reliable,lyric_language_reliable,NLP_sentiment,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness,from_genius,from_lyrics_wiki,NLP_emotion_anger,NLP_emotion_fear,NLP_emotion_joy,NLP_emotion_sadness,NLP_emotion_surprise,trial_0.0,trial_1.0,trial_2.0,trial_3.0,try_idx_0.0,try_idx_1.0,try_idx_2.0,try_idx_3.0,try_idx_4.0,try_idx_5.0,NLP_topic_poetry,NLP_topic_music,NLP_topic_drugs,NLP_topic_romance,NLP_topic_parenting,NLP_topic_running,NLP_topic_entertainment_news,NLP_topic_military,NLP_topic_fashion,NLP_topic_cricket,NLP_topic_swimming,NLP_topic_surfing,NLP_topic_biology,NLP_topic_guns,NLP_topic_personal,NLP_topic_history,NLP_topic_books,NLP_topic_sailing,NLP_topic_bicycling,NLP_topic_business,NLP_topic_film,NLP_topic_fiction,NLP_topic_golf,NLP_topic_christianity,NLP_topic_comics,NLP_topic_nostalgia,NLP_topic_weight_training,NLP_topic_astronomy,NLP_topic_art,NLP_topic_hunting,lyric_language_english,lyric_language_spanish,lyric_language_korean,lyric_language_french,lyric_language_tagalog,lyric_language_german,lyric_language_portuguese,lyric_language_russian,lyric_language_italian,track_name_language_english,track_name_language_spanish
0,0.011400,0.0,0.775,0.0,351640,0.0,0.0,0.0,0.828,0.0,0.0,0.0,0.0,0.367000,0.0,11.0,0.0,0.2460,-7.511,0.0,0.0,0.0,391407,0.0,8.0,0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0503,108.016,4.0,7.0,0.531,2010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.921551,0.545398,0.455326,0.594833,0.475259,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,0.094400,0.0,0.591,0.0,213333,0.0,0.0,0.0,0.892,0.0,0.0,0.0,0.0,0.000000,0.0,8.0,0.0,0.1130,-6.114,0.0,0.0,0.0,391407,0.0,47.0,1,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.2930,89.849,4.0,10.0,0.712,2012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.927271,0.573924,0.500366,0.617451,0.423998,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0.312000,0.0,0.496,0.0,309346,0.0,0.0,0.0,0.531,0.0,0.0,0.0,0.0,0.009010,0.0,0.0,0.0,0.1130,-9.889,0.0,0.0,0.0,391407,0.0,15.0,2,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0325,119.159,4.0,10.0,0.507,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.352308,0.532170,0.433947,0.650911,0.457791,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0.000029,0.0,0.420,0.0,286466,0.0,0.0,0.0,0.914,0.0,0.0,0.0,0.0,0.028200,0.0,6.0,0.0,0.2200,-4.322,0.0,0.0,0.0,391407,0.0,55.0,3,0.0,0.0,20.0,1.0,0.0,0.0,0.0,0.0431,170.199,4.0,10.0,0.301,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.975505,0.522853,0.438368,0.676775,0.484940,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,0.024500,0.0,0.633,0.0,196848,0.0,0.0,0.0,0.881,0.0,0.0,0.0,0.0,0.000147,0.0,1.0,0.0,0.0474,-5.570,0.0,0.0,0.0,391407,0.0,54.0,4,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.1540,87.920,4.0,10.0,0.622,2014.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.983508,0.516442,0.353284,0.735746,0.483784,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [25]:
playlists_df_clean.describe()

,acousticness,country,danceability,disco,duration_ms,edm,electronic,emo,energy,folk,funk,gospel,house,instrumentalness,k-pop,key,latin,liveness,loudness,metal,metalcore,mode,pid,pop,popularity,pos,punk,reggaeton,related_artist_n,rock,rockabilly,singer-songwriter,soul,speechiness,tempo,time_signature,top_track_n,valence,year,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language_reliable,lyric_language_reliable,NLP_sentiment,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness,from_genius,from_lyrics_wiki,NLP_emotion_anger,NLP_emotion_fear,NLP_emotion_joy,NLP_emotion_sadness,NLP_emotion_surprise,trial_0.0,trial_1.0,trial_2.0,trial_3.0,try_idx_0.0,try_idx_1.0,try_idx_2.0,try_idx_3.0,try_idx_4.0,try_idx_5.0,NLP_topic_poetry,NLP_topic_music,NLP_topic_drugs,NLP_topic_romance,NLP_topic_parenting,NLP_topic_running,NLP_topic_entertainment_news,NLP_topic_military,NLP_topic_fashion,NLP_topic_cricket,NLP_topic_swimming,NLP_topic_surfing,NLP_topic_biology,NLP_topic_guns,NLP_topic_personal,NLP_topic_history,NLP_topic_books,NLP_topic_sailing,NLP_topic_bicycling,NLP_topic_business,NLP_topic_film,NLP_topic_fiction,NLP_topic_golf,NLP_topic_christianity,NLP_topic_comics,NLP_topic_nostalgia,NLP_topic_weight_training,NLP_topic_astronomy,NLP_topic_art,NLP_topic_hunting,lyric_language_english,lyric_language_spanish,lyric_language_korean,lyric_language_french,lyric_language_tagalog,lyric_language_german,lyric_language_portuguese,lyric_language_russian,lyric_language_italian,track_name_language_english,track_name_language_spanish
count,55000.000000,55000.000000,55000.000000,55000.000000,5.500000e+04,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,54999.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,54995.000000,55000.000000,54999.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,52007.000000,51649.000000,47546.000000,47546.000000,47546.000000,47546.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000
mean,0.257011,0.031745,0.596193,0.008545,2.350665e+05,0.045982,0.007636,0.015345,0.632996,0.011345,0.015473,0.003509,0.009891,0.074761,0.004673,5.206436,0.040345,0.190331,-7.525920,0.004564,0.005182,0.653818,497957.939000,0.291127,41.108784,27.000000,0.007600,0.020982,19.883436,0.076909,0.004891,0.013527,0.015945,0.091508,121.494737,3.930564,9.852059,0.483090,2007.687376,0.112091,0.060345,0.042509,0.034800,0.027182,0.021364,0.020418,0.017036,0.014873,0.014000,0.543818,0.992924,0.674081,0.563834,0.431771,0.665134,0.456555,0.965673,0.002891,0.069982,0.041545,0.170145,0.543400,0.039400,0.871709,0.023909,0.068836,0.004109,0.880764,0.084109,0.000436,0.000236,0.000109,0.002909,0.306564,0.215273,0.046527,0.042582,0.031891,0.016855,0.013964,0.012073,0.008927,0.005909,0.005691,0.005600,0.005382,0.0

### [1] Mean imputation

In [26]:
# mean imputation

mean_imputation_cols = ['year', 'NLP_sentiment', 'NLP_personality_extraversion', 'NLP_personality_openness',
                        'NLP_personality_agreeableness', 'NLP_personality_conscientiousness']

for col in mean_imputation_cols:
    nan_idx = playlists_df_clean[playlists_df_clean[col].isna()].index.values
#     print(nan_idx)
    
    for idx in nan_idx:
        playlists_df_clean.at[idx,col] = playlists_df_clean[col].mean()
    
#     print(playlists_df_clean[col].mean())

### [2] Filling zero

In [27]:
np.sum(playlists_df_clean.isna())[np.sum(playlists_df_clean.isna()).values != 0]

popularity                    1
top_track_n                   5
lyric_language_reliable    2993
dtype: int64

In [28]:
playlists_df_clean = playlists_df_clean.fillna(0)

In [29]:
display(playlists_df_clean.head())
display(playlists_df_clean.describe())
print('Shape : ', playlists_df_clean.shape)

,acousticness,country,danceability,disco,duration_ms,edm,electronic,emo,energy,folk,funk,gospel,house,instrumentalness,k-pop,key,latin,liveness,loudness,metal,metalcore,mode,pid,pop,popularity,pos,punk,reggaeton,related_artist_n,rock,rockabilly,singer-songwriter,soul,speechiness,tempo,time_signature,top_track_n,valence,year,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language_reliable,lyric_language_reliable,NLP_sentiment,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness,from_genius,from_lyrics_wiki,NLP_emotion_anger,NLP_emotion_fear,NLP_emotion_joy,NLP_emotion_sadness,NLP_emotion_surprise,trial_0.0,trial_1.0,trial_2.0,trial_3.0,try_idx_0.0,try_idx_1.0,try_idx_2.0,try_idx_3.0,try_idx_4.0,try_idx_5.0,NLP_topic_poetry,NLP_topic_music,NLP_topic_drugs,NLP_topic_romance,NLP_topic_parenting,NLP_topic_running,NLP_topic_entertainment_news,NLP_topic_military,NLP_topic_fashion,NLP_topic_cricket,NLP_topic_swimming,NLP_topic_surfing,NLP_topic_biology,NLP_topic_guns,NLP_topic_personal,NLP_topic_history,NLP_topic_books,NLP_topic_sailing,NLP_topic_bicycling,NLP_topic_business,NLP_topic_film,NLP_topic_fiction,NLP_topic_golf,NLP_topic_christianity,NLP_topic_comics,NLP_topic_nostalgia,NLP_topic_weight_training,NLP_topic_astronomy,NLP_topic_art,NLP_topic_hunting,lyric_language_english,lyric_language_spanish,lyric_language_korean,lyric_language_french,lyric_language_tagalog,lyric_language_german,lyric_language_portuguese,lyric_language_russian,lyric_language_italian,track_name_language_english,track_name_language_spanish
0,0.011400,0.0,0.775,0.0,351640,0.0,0.0,0.0,0.828,0.0,0.0,0.0,0.0,0.367000,0.0,11.0,0.0,0.2460,-7.511,0.0,0.0,0.0,391407,0.0,8.0,0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0503,108.016,4.0,7.0,0.531,2010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.921551,0.545398,0.455326,0.594833,0.475259,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,0.094400,0.0,0.591,0.0,213333,0.0,0.0,0.0,0.892,0.0,0.0,0.0,0.0,0.000000,0.0,8.0,0.0,0.1130,-6.114,0.0,0.0,0.0,391407,0.0,47.0,1,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.2930,89.849,4.0,10.0,0.712,2012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.927271,0.573924,0.500366,0.617451,0.423998,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0.312000,0.0,0.496,0.0,309346,0.0,0.0,0.0,0.531,0.0,0.0,0.0,0.0,0.009010,0.0,0.0,0.0,0.1130,-9.889,0.0,0.0,0.0,391407,0.0,15.0,2,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0325,119.159,4.0,10.0,0.507,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.352308,0.532170,0.433947,0.650911,0.457791,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0.000029,0.0,0.420,0.0,286466,0.0,0.0,0.0,0.914,0.0,0.0,0.0,0.0,0.028200,0.0,6.0,0.0,0.2200,-4.322,0.0,0.0,0.0,391407,0.0,55.0,3,0.0,0.0,20.0,1.0,0.0,0.0,0.0,0.0431,170.199,4.0,10.0,0.301,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.975505,0.522853,0.438368,0.676775,0.484940,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,0.024500,0.0,0.633,0.0,196848,0.0,0.0,0.0,0.881,0.0,0.0,0.0,0.0,0.000147,0.0,1.0,0.0,0.0474,-5.570,0.0,0.0,0.0,391407,0.0,54.0,4,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.1540,87.920,4.0,10.0,0.622,2014.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.983508,0.516442,0.353284,0.735746,0.483784,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


,acousticness,country,danceability,disco,duration_ms,edm,electronic,emo,energy,folk,funk,gospel,house,instrumentalness,k-pop,key,latin,liveness,loudness,metal,metalcore,mode,pid,pop,popularity,pos,punk,reggaeton,related_artist_n,rock,rockabilly,singer-songwriter,soul,speechiness,tempo,time_signature,top_track_n,valence,year,is_top_track_0,is_top_track_1,is_top_track_2,is_top_track_3,is_top_track_4,is_top_track_5,is_top_track_6,is_top_track_7,is_top_track_8,is_top_track_9,track_name_language_reliable,lyric_language_reliable,NLP_sentiment,NLP_personality_extraversion,NLP_personality_openness,NLP_personality_agreeableness,NLP_personality_conscientiousness,from_genius,from_lyrics_wiki,NLP_emotion_anger,NLP_emotion_fear,NLP_emotion_joy,NLP_emotion_sadness,NLP_emotion_surprise,trial_0.0,trial_1.0,trial_2.0,trial_3.0,try_idx_0.0,try_idx_1.0,try_idx_2.0,try_idx_3.0,try_idx_4.0,try_idx_5.0,NLP_topic_poetry,NLP_topic_music,NLP_topic_drugs,NLP_topic_romance,NLP_topic_parenting,NLP_topic_running,NLP_topic_entertainment_news,NLP_topic_military,NLP_topic_fashion,NLP_topic_cricket,NLP_topic_swimming,NLP_topic_surfing,NLP_topic_biology,NLP_topic_guns,NLP_topic_personal,NLP_topic_history,NLP_topic_books,NLP_topic_sailing,NLP_topic_bicycling,NLP_topic_business,NLP_topic_film,NLP_topic_fiction,NLP_topic_golf,NLP_topic_christianity,NLP_topic_comics,NLP_topic_nostalgia,NLP_topic_weight_training,NLP_topic_astronomy,NLP_topic_art,NLP_topic_hunting,lyric_language_english,lyric_language_spanish,lyric_language_korean,lyric_language_french,lyric_language_tagalog,lyric_language_german,lyric_language_portuguese,lyric_language_russian,lyric_language_italian,track_name_language_english,track_name_language_spanish
count,55000.000000,55000.000000,55000.000000,55000.000000,5.500000e+04,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000
mean,0.257011,0.031745,0.596193,0.008545,2.350665e+05,0.045982,0.007636,0.015345,0.632996,0.011345,0.015473,0.003509,0.009891,0.074761,0.004673,5.206436,0.040345,0.190331,-7.525920,0.004564,0.005182,0.653818,497957.939000,0.291127,41.108036,27.000000,0.007600,0.020982,19.883436,0.076909,0.004891,0.013527,0.015945,0.091508,121.494737,3.930564,9.851164,0.483090,2007.687376,0.112091,0.060345,0.042509,0.034800,0.027182,0.021364,0.020418,0.017036,0.014873,0.014000,0.543818,0.938891,0.674081,0.563834,0.431771,0.665134,0.456555,0.965673,0.002891,0.069982,0.041545,0.170145,0.543400,0.039400,0.871709,0.023909,0.068836,0.004109,0.880764,0.084109,0.000436,0.000236,0.000109,0.002909,0.306564,0.215273,0.046527,0.042582,0.031891,0.016855,0.013964,0.012073,0.008927,0.005909,0.005691,0.005600,0.005382,0.0

Shape :  (55000, 114)


In [30]:
np.sum(playlists_df_clean.isna())[np.sum(playlists_df_clean.isna()).values != 0]

Series([], dtype: int64)

In [31]:
playlists_df_clean.to_pickle('playlists_df_clean.pkl')